# Task
using pytoch build model

Here is all the data you need:
"fashion-mnist_train.csv"

## Data loading

### Subtask:
Load the Fashion-MNIST training data from the provided CSV file.


**Reasoning**:
I need to load the data from the provided CSV file into a pandas DataFrame and inspect the first few rows to verify the data.



In [1]:
import pandas as pd

try:
    df = pd.read_csv('fashion-mnist_train.csv')
    display(df.head())
except FileNotFoundError:
    print("Error: 'fashion-mnist_train.csv' not found. Please ensure the file exists in the current directory.")
    df = None
except pd.errors.ParserError:
    print("Error: Could not parse the CSV file. Please check the file format.")
    df = None
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    df = None

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data preparation

### Subtask:
Prepare the data for model training.


**Reasoning**:
Convert the data to PyTorch tensors, normalize pixel values, one-hot encode labels, and split the data into training and validation sets.



In [2]:
import torch
from torch.utils.data import random_split, TensorDataset

# Extract pixel data and labels
X = df.drop('label', axis=1).values
y = df['label'].values

# Normalize pixel values
X = X / 255.0

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# One-hot encode the labels
y_onehot = torch.nn.functional.one_hot(y, num_classes=10)

# Split data
dataset = TensorDataset(X, y_onehot)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
X_train, X_val, y_train, y_val = random_split(dataset, [train_size, val_size])

X_train = X_train[:][0]
y_train = X_train[:][1]
X_val = X_val[:][0]
y_val = X_val[:][1]

ValueError: not enough values to unpack (expected 4, got 2)

**Reasoning**:
The previous code had an error in unpacking the results of `random_split`.  `random_split` returns two subsets of the dataset, not four.  I will correct the unpacking and then access the data and labels correctly from the resulting subsets.



In [3]:
import torch
from torch.utils.data import random_split, TensorDataset

# Extract pixel data and labels
X = df.drop('label', axis=1).values
y = df['label'].values

# Normalize pixel values
X = X / 255.0

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# One-hot encode the labels
y_onehot = torch.nn.functional.one_hot(y, num_classes=10)

# Split data
dataset = TensorDataset(X, y_onehot)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Access data and labels correctly
X_train = train_dataset[:][0]
y_train = train_dataset[:][1]
X_val = val_dataset[:][0]
y_val = val_dataset[:][1]

## Model training

### Subtask:
Define and train a Convolutional Neural Network (CNN) model using PyTorch.


**Reasoning**:
Define the CNN model, set up training parameters, train the model, and save the trained model.



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define the CNN model
class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super(FashionMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 64)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = FashionMNISTCNN()

# Set up training parameters
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss with one-hot encoded labels
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
batch_size = 64

# Create data loaders
train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        labels = torch.argmax(labels, dim=1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'fashion_mnist_cnn.pth')

Epoch [1/10], Step [100/750], Loss: 0.7413
Epoch [1/10], Step [200/750], Loss: 0.4799
Epoch [1/10], Step [300/750], Loss: 0.2945
Epoch [1/10], Step [400/750], Loss: 0.4719
Epoch [1/10], Step [500/750], Loss: 0.2767
Epoch [1/10], Step [600/750], Loss: 0.3457
Epoch [1/10], Step [700/750], Loss: 0.4818
Epoch [2/10], Step [100/750], Loss: 0.2701
Epoch [2/10], Step [200/750], Loss: 0.3668
Epoch [2/10], Step [300/750], Loss: 0.3619
Epoch [2/10], Step [400/750], Loss: 0.2706
Epoch [2/10], Step [500/750], Loss: 0.4210
Epoch [2/10], Step [600/750], Loss: 0.2072
Epoch [2/10], Step [700/750], Loss: 0.2397
Epoch [3/10], Step [100/750], Loss: 0.2355
Epoch [3/10], Step [200/750], Loss: 0.2085
Epoch [3/10], Step [300/750], Loss: 0.2198
Epoch [3/10], Step [400/750], Loss: 0.3081
Epoch [3/10], Step [500/750], Loss: 0.4181
Epoch [3/10], Step [600/750], Loss: 0.4075
Epoch [3/10], Step [700/750], Loss: 0.3348
Epoch [4/10], Step [100/750], Loss: 0.2814
Epoch [4/10], Step [200/750], Loss: 0.1408
Epoch [4/10

## Model evaluation

### Subtask:
Evaluate the trained CNN model on the validation set.


**Reasoning**:
Evaluate the trained CNN model on the validation set. This involves loading the saved model, creating a DataLoader for the validation set, setting the model to evaluation mode, iterating through the validation data, performing a forward pass, calculating the accuracy, and printing the result.



In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Define the CNN model (same as in the training step)
class FashionMNISTCNN(nn.Module):
    def __init__(self):
        super(FashionMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 64)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the saved model
model = FashionMNISTCNN()
model.load_state_dict(torch.load('fashion_mnist_cnn.pth'))

# Create a DataLoader for the validation dataset
batch_size = 64
val_loader = DataLoader(list(zip(X_val, y_val)), batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Calculate the accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        labels = torch.argmax(labels, dim=1) # Get the class index from one-hot encoding
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 91.75%


## Summary:

### 1. Q&A

* **What is the validation accuracy of the trained CNN model?** The trained CNN model achieved a validation accuracy of 91.75%.

### 2. Data Analysis Key Findings

* **Data Preparation:** Pixel values were normalized to the range [0, 1] by dividing by 255.0. Labels were one-hot encoded using PyTorch's `one_hot` function. The data was split into an 80/20 training and validation sets.
* **Model Training:**  The model was trained for 10 epochs with a batch size of 64, using the Adam optimizer with a learning rate of 0.001 and the CrossEntropyLoss function.
* **Model Evaluation:** The trained model achieved a validation accuracy of 91.75%.

### 3. Insights or Next Steps

* **Hyperparameter Tuning:** Experiment with different hyperparameters (learning rate, batch size, number of epochs, network architecture) to potentially improve the model's performance.
* **Further Evaluation:**  Evaluate the model on a held-out test set to obtain a more robust estimate of its generalization performance.
